In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
from multiprocessing import shared_memory

flag=np.ndarray([0])
flagshm= shared_memory.SharedMemory(name= 'wnsm_601d9d10')#flag 주소
flag = np.ndarray(flag.shape, dtype=flag.dtype, buffer=flagshm.buf)


frameshm= shared_memory.SharedMemory(name= 'wnsm_5c23f5ec')#frame주소
cap = cv2.VideoCapture('./2023-09-05#17h42m12s.mp4')
ret, frame = cap.read() #동영상 정보 읽어오기 -read
frame = np.ndarray(frame.shape, dtype=frame.dtype, buffer=frameshm.buf)

def getIntersectionArea(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    return (xB - xA + 1) * (yB - yA + 1)

cap = cv2.VideoCapture(0) # 삭제해도 됨
coord_list = []

person_model = YOLO("./weights/person.pt")
door_model = YOLO("./weights/door.pt")

while True:
    #ret, frame = cap.read() # 여기서 프레임을 받아옴
    if ret:
        frame = cv2.flip(frame, 1) # 삭제해도 됨
        coord_list = []
        for coord in person_model.predict(frame)[0].boxes:
            x, y, w, h = coord.xywh[0][:]
            xmin, ymin, xmax, ymax = int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
            cv2.rectangle(frame, (xmin, ymin, xmax, ymax), (0, 0, 255), 4) # 삭제해도 됨
            coord_list.append([xmin, ymin, xmax, ymax])
            break
            
        for coord in door_model.predict(frame)[0].boxes:
            x, y, w, h = coord.xywh[0][:]
            xmin, ymin, xmax, ymax = int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 4) # 삭제해도 됨
            coord_list.append([xmin, ymin, xmax, ymax])
        
        distance = 0
        for i in range(len(coord_list) - 1): 
            for j in range(i+1, len(coord_list)):
                area = getIntersectionArea(coord_list[i], coord_list[j])
            if area > 0:
                # 여기서 신호 보내면 됨
                flag=2
                cv2.putText(frame, "WARNING!!", (50, 100), 1, 3, (0, 0, 255), 5, cv2.LINE_AA) # 삭제해도 됨
                break
            else:
                flag=0
        cv2.imshow("frame", frame) # 삭제해도 됨
    
        if cv2.waitKey(10) == 27: # 삭제해도 됨
            break # 삭제해도 됨
    
cap.release() # 삭제해도 됨
cv2.destroyAllWindows() # 삭제해도 됨


0: 384x640 1 Person, 86.8ms
Speed: 3.0ms preprocess, 86.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 80.0ms
Speed: 4.0ms preprocess, 80.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
cap.release()
cv2.destroyAllWindows()